### 0. Imports and requirements

* В данном соревновании мы имеем дело с последовательностями, один из интуитивных способов работы с ними - использование рекуррентных сетей. Данный бейзлайн посвящен тому, чтобы показать, как можно строить хорошие решения без использования сложного и трудоемкого feature engineering-а (чтобы эффективно решать ту же задачу с высоким качеством с помощью бустингов нужно несколько тысяч признаков), благодаря рекуррентным сетям. В этом ноутбуке мы построим продвинутое решение с использованием фреймфорка `torch`. Для комфортной работы Вам понадобится машина с `GPU` (хватит ресурсов `google colab` или `kaggle`).

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
pd.set_option('display.max_columns', None)

# добавим корневую папку, в ней лежат все необходимые полезные функции для обработки данных
sys.path.append('../../')
sys.path.append('../')

### 1. Data Preprocessing

* В базовом решении на нейронных сетях приведена вся обработка данных и все стадии препроцесснга. В данном ноутбуке опустим данный раздел и используем уже готовые данные.

In [2]:
path_to_dataset = '../../../val_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

['../../../val_buckets/processed_chunk_000.pkl',
 '../../../val_buckets/processed_chunk_001.pkl',
 '../../../val_buckets/processed_chunk_002.pkl',
 '../../../val_buckets/processed_chunk_003.pkl',
 '../../../val_buckets/processed_chunk_004.pkl']

In [3]:
path_to_dataset = '../../../train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_train

['../../../train_buckets/processed_chunk_000.pkl',
 '../../../train_buckets/processed_chunk_001.pkl',
 '../../../train_buckets/processed_chunk_002.pkl',
 '../../../train_buckets/processed_chunk_003.pkl',
 '../../../train_buckets/processed_chunk_004.pkl',
 '../../../train_buckets/processed_chunk_005.pkl',
 '../../../train_buckets/processed_chunk_006.pkl',
 '../../../train_buckets/processed_chunk_007.pkl',
 '../../../train_buckets/processed_chunk_008.pkl',
 '../../../train_buckets/processed_chunk_009.pkl']

### 2. Modeling

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


* Для создания модели будем использовать фреймворк `torch`. В нем есть все, чтобы писать произвольные сложные архитектуры и быстро эксперементировать. Для того, чтобы мониторить и логировать весь процесс во время обучения сетей, рекомендуется использовать надстройки над данным фреймворков, например, `lightning`.

* В бейзлайне мы предлагаем базовые компоненты, чтобы можно было обучать нейронную сеть и отслеживать ее качество. Для этого вам предоставлены следующие функции:
    * `data_generators.batches_generator` - функция-генератор, итеративно возвращает батчи, поддерживает батчи для `tensorflow.keras` и `torch.nn.module` моделей. В зависимости от флага `is_train` может быть использована для генерации батчей на train/val/test стадию.
    * функция `pytorch_training.train_epoch` - обучает модель одну эпоху.
    * функция `pytorch_training.eval_model` - проверяет качество модели на отложенной выборке и возвращает roc_auc_score.
    * функция `pytorch_training.inference` - делает предикты на новых данных и готовит фрейм для проверяющей системы.
    * класс `training_aux.EarlyStopping` - реализует early_stopping, сохраняя лучшую модель. Пример использования приведен ниже.

In [5]:
from data_generators import batches_generator, transaction_features
from pytorch_training import train_epoch, eval_model, inference
from training_aux import EarlyStopping

* Все признаки в нашей модели будут категориальными. Для их представления в модели используем категориальные эмбеддинги. Для этого нужно каждому категориальному признаку задать размерность латентного пространства. Используем [формулу](https://forums.fast.ai/t/size-of-embedding-for-categorical-variables/42608) из библиотеки `fast.ai`. Все отображения хранятся в файле `embedding_projections.pkl`

In [6]:
with open('../constants/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)

* Реализуем модель. Все входные признаки представим в виде эмбеддингов, сконкатенируем, чтобы получить векторное представление транзакции. Используем SpatialDropout, чтобы регуляризовать эмбеддинги. Подадим последовательности в `BiGRU` рекуррентную сеть. Используем все скрытые состояния сети, чтобы получить агрегированное представление об истории транзакции - пропустим все скрытые состояния `BiGRU` через `AvgPooling` и черерз `MaxPooling`. Представим признак `product` в виде отдельного эмбеддинга. Сконкатенируем его с результатами пулингов. На основе такого входа построим небольшой `MLP`, выступающий классификатором для целевой задачи. Используем градиентный спуск, чтобы решить оптимизационную задачу.

In [7]:
class TransactionsRnn(nn.Module):
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', 
                 rnn_units=128, top_classifier_units=32):
        super(TransactionsRnn, self).__init__()
        
        self._transaction_cat_embeddings = nn.ModuleList([self._create_embedding_projection(*embedding_projections[feature], 
                                                                                            padding_idx=None) 
                                                          for feature in transactions_cat_features])
        self._spatial_dropout = nn.Dropout2d(0.05)
        self._transaction_cat_embeddings_concated_dim = sum([embedding_projections[x][1] for x in transactions_cat_features])
        
        self._product_embedding = self._create_embedding_projection(*embedding_projections[product_col_name], padding_idx=None)
        
        self._gru = nn.GRU(input_size=self._transaction_cat_embeddings_concated_dim,
                             hidden_size=rnn_units, batch_first=True, bidirectional=True)
        
        self._hidden_size = rnn_units
        
        # построим классификатор, он будет принимать на вход: 
        # [max_pool(gru_states), avg_pool(gru_states), product_embed]
        pooling_result_dimension = self._hidden_size * 2
         
        self._top_classifier = nn.Sequential(nn.Linear(in_features=2*pooling_result_dimension + 
                                                       embedding_projections[product_col_name][1], 
                                                       out_features=top_classifier_units),
                                             nn.ReLU(),
                                             nn.Linear(in_features=top_classifier_units, out_features=1)
                                            )
        
    def forward(self, transactions_cat_features, product_feature):
        batch_size = product_feature.shape[0]
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        concated_embeddings = torch.cat(embeddings, dim=-1)
        concated_embeddings = concated_embeddings.permute(0, 2, 1).unsqueeze(3)
        
        dropout_embeddings = self._spatial_dropout(concated_embeddings)
        dropout_embeddings = dropout_embeddings.squeeze(3).permute(0, 2, 1)

        states, _ = self._gru(dropout_embeddings)
        
        rnn_max_pool = states.max(dim=1)[0]
        rnn_avg_pool = states.sum(dim=1) / states.shape[1]        
        
        product_embed = self._product_embedding(product_feature)
                
        combined_input = torch.cat([rnn_max_pool, rnn_avg_pool, product_embed], dim=-1)
            
        logit = self._top_classifier(combined_input)        
        return logit
    
    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(num_embeddings=cardinality+add_missing, embedding_dim=embed_size, padding_idx=padding_idx)


### 3. Training

In [8]:
! mkdir ../../rnn_baseline/checkpoints/

mkdir: cannot create directory ‘../../rnn_baseline/checkpoints/’: File exists


In [9]:
! rm -r ../../rnn_baseline/checkpoints/pytorch_advanced_baseline
! mkdir ../../rnn_baseline/checkpoints/pytorch_advanced_baseline

* Для того, чтобы детектировать переобучение используем EarlyStopping.

In [10]:
path_to_checkpoints = '../../rnn_baseline/checkpoints/pytorch_advanced_baseline/'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='torch')

In [11]:
num_epochs = 15
train_batch_size = 128
val_batch_szie = 128

In [12]:
model = TransactionsRnn(transaction_features, embedding_projections, top_classifier_units=128).to(device)

In [13]:
model

TransactionsRnn(
  (_transaction_cat_embeddings): ModuleList(
    (0): Embedding(12, 6)
    (1): Embedding(8, 5)
    (2): Embedding(176, 29)
    (3): Embedding(23, 9)
    (4): Embedding(5, 3)
    (5): Embedding(4, 3)
    (6): Embedding(8, 5)
    (7): Embedding(4, 3)
    (8): Embedding(109, 22)
    (9): Embedding(25, 9)
    (10): Embedding(164, 28)
    (11): Embedding(29, 10)
    (12): Embedding(8, 5)
    (13): Embedding(25, 9)
    (14): Embedding(54, 15)
    (15): Embedding(11, 6)
    (16): Embedding(24, 9)
    (17): Embedding(11, 6)
  )
  (_spatial_dropout): Dropout2d(p=0.15, inplace=False)
  (_product_embedding): Embedding(6, 4)
  (_gru): GRU(182, 128, batch_first=True, bidirectional=True)
  (_top_classifier): Sequential(
    (0): Linear(in_features=516, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [14]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())

* Запустим цикл обучения, каждую эпоху будем логировать лосс, а так же roc-auc на валидации и на обучении. Будем сохрнаять веса после каждой эпохи, а так же лучшие с помощью early_stopping.

In [15]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    train_epoch(model, optimizer, dataset_train, batch_size=train_batch_size, 
                shuffle=True, print_loss_every_n_batches=500, device=device)
    
    val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie, device=device)
    es(val_roc_auc, model)
    
    train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
    print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')
    
    if es.early_stop:
        print('Early stopping reached. Stop training...')
        break
        
    torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))

Starting epoch 1


Training loss after 7000 batches: 0.11595076322555542



Validation ROC-AUC improved (-inf --> 0.777086).  Saving model ...



Epoch 1 completed. Train roc-auc: 0.7777762280147325, Val roc-auc: 0.7770862233891969
Starting epoch 2


Training loss after 7000 batches: 0.11137688159942627


...


No imporvement in Validation ROC-AUC. Current: 0.795061. Current best: 0.796498
EarlyStopping counter: 2 out of 3



Epoch 10 completed. Train roc-auc: 0.8681974270545914, Val roc-auc: 0.7950609452692265
Starting epoch 11


Training loss after 7000 batches: 0.09896302223205566



No imporvement in Validation ROC-AUC. Current: 0.791978. Current best: 0.796498
EarlyStopping counter: 3 out of 3



Epoch 11 completed. Train roc-auc: 0.8728909784116261, Val roc-auc: 0.7919782048146008
Early stopping reached. Stop training...


### 4. Submission

* Все готово, чтобы сделать предсказания для тестовой выборки. Нужно только подготовить данные в том же формате, как и для train. В ноутбуке с базовым решением приведен весь процесс предобработки тестовых данных.

In [16]:
path_to_test_dataset = '../../../test_buckets/'
dir_with_test_datasets = os.listdir(path_to_test_dataset)
dataset_test = sorted([os.path.join(path_to_test_dataset, x) for x in dir_with_test_datasets])

dataset_test

['../../../test_buckets/processed_chunk_000.pkl',
 '../../../test_buckets/processed_chunk_001.pkl',
 '../../../test_buckets/processed_chunk_002.pkl',
 '../../../test_buckets/processed_chunk_003.pkl',
 '../../../test_buckets/processed_chunk_004.pkl']

* Отдельный вопрос, какую из построенных моделей использовать для того, чтобы делать предсказания на тест. Можно выбирать лучшую по early_stopping. В таком случае есть риск, что мы подгонимся под валидационную выборку, особенно если она не является очень репрезентативной, однако это самый базовый вариант (используем его). Можно делать разные версии ансамблирования, используя веса с разных эпох. Такой подход требует дополнительного кода (обязательно попробуйте его!). Наконец, можно выбирать такую модель, которая показывает хорошие результаты на валидации и в то же время, не слишком переучена под train выборку.

In [17]:
! ls $path_to_checkpoints

best_checkpoint.pt     epoch_3_val_0.771.pt  epoch_7_val_0.794.pt
epoch_10_val_0.795.pt  epoch_4_val_0.793.pt  epoch_8_val_0.796.pt
epoch_1_val_0.777.pt   epoch_5_val_0.795.pt  epoch_9_val_0.792.pt
epoch_2_val_0.781.pt   epoch_6_val_0.794.pt


In [18]:
model.load_state_dict(torch.load(os.path.join(path_to_checkpoints, 'best_checkpoint.pt')))

<All keys matched successfully>

In [19]:
test_preds = inference(model, dataset_test, batch_size=128, device=device)

In [21]:
test_preds.head()

,app_id,score
0,1063655,-3.908171
1,1063672,-2.741681
2,1063694,-5.014806
3,1063709,-2.887169
4,1063715,-3.876974


In [26]:
test_preds.to_csv('rnn_advanced_baseline_submission.csv', index=None) # ~ 0.760 на public test